<a href="https://colab.research.google.com/github/ClesteA/MedSim/blob/main/medsim_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()

# 1. Unsloth'u Colab için özel parametreyle kuruyoruz
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# 2. Xformers ve diğerlerini derleme yapmadan (no-deps) kuruyoruz
# Bu komut pip'in "dur ben bunu derleyeyim" demesini engeller.
!pip install --no-deps xformers trl peft accelerate bitsandbytes

# 3. Gereksiz uyarıları susturmak için ek paketler
!pip install --no-deps packaging ninja einops

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Ne kadar uzun metin okuyabileceği
dtype = None # None yapınca otomatik algılar (Float16)
load_in_4bit = True # 4-bit quantization (Hız ve hafıza için şart)

# Llama 3.1 8B Instruct modelini yüklüyoruz
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Modele LoRA adaptörlerini ekliyoruz (Eğitilebilir hale getiriyoruz)
# Modeli yüklediğiniz kod bloğunda şu ayarları güncelleyin:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # ÖNCEKİ MUHTEMELEN 16 İDİ. Bunu 64 yapın (Kapasiteyi 4 katına çıkarır).
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Unsloth 2025.11.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
import json
from datasets import Dataset

# Dosya adını buraya yaz (Colab'e yüklediğin dosya)
DOSYA_ADI = "/content/medsim_genis_db_v2.json"

# Veriyi oku
try:
    with open(DOSYA_ADI, 'r', encoding='utf-8') as f:
        ham_veri = json.load(f)
    print(f"{len(ham_veri)} adet vaka bulundu. İşleniyor...")
except FileNotFoundError:
    print("HATA: Dosya bulunamadı! Lütfen sol menüden dosyayı yükleyin.")
    ham_veri = []

# Alpaca Prompt Formatı (Modelin instruction'ı anlaması için standart kalıp)
alpaca_prompt = """Aşağıda bir görevi tanımlayan bir talimat ve bağlam sağlayan bir girdi bulunmaktadır. İsteği uygun şekilde tamamlayan bir yanıt yazın.

### Talimat:
{}

### Girdi:
{}

### Yanıt:
{}"""

instruction_text = "Sen uzman bir dahiliye doktorusun. Aşağıdaki hasta bilgilerine dayanarak tanıyı ve bulguları JSON formatında analiz et. Ayrıca Türkçe dil yeteneklerini de geliştir."
EOS_TOKEN = tokenizer.eos_token

formatted_data = []

for vaka in ham_veri:
    # Girdi: Hasta bilgileri
    input_text = f"Hasta: {vaka['hasta_kimlik']['ad_soyad']}, {vaka['hasta_kimlik']['yas']} yaşında, {vaka['hasta_kimlik']['meslek']}.\nŞikayet: {vaka['hasta_kimlik']['sikayet']}"

    # Çıktı: Modelin öğrenmesini istediğimiz JSON cevabı
    output_text = json.dumps({
        "tani": vaka['gizli_tani'],
        "anamnez": vaka['anamnez'],
        "bulgular": vaka['bulgular']
    }, ensure_ascii=False)

    # Formatı birleştir
    text = alpaca_prompt.format(instruction_text, input_text, output_text) + EOS_TOKEN
    formatted_data.append({"text": text})

# HuggingFace Dataset formatına çevir
dataset = Dataset.from_list(formatted_data)
print("Veri seti eğitime hazır!")

513 adet vaka bulundu. İşleniyor...
Veri seti eğitime hazır!


In [ ]:
from transformers import TrainerCallback, TrainingArguments # TrainingArguments eklendi
from trl import SFTTrainer # SFTTrainer eklendi
from unsloth import is_bfloat16_supported # is_bfloat16_supported eklendi

# --- 1. LOSS KONTROL MEKANİZMASI (FREN SİSTEMİ) ---
class TargetLossCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs): # **kwargs eklendi
        if logs and 'loss' in logs:
            current_loss = logs['loss']
            # Loss 0.38'in altına inerse tehlike çanları çalar, durdururuz.
            # 0.4 hedeflediğimiz için 0.38 güvenli alt sınırdır.
            if current_loss < 0.38:
                print(f"\n✋ HEDEF LOSS'A ULAŞILDI: {current_loss:.4f}. Ezberi önlemek için durduruluyor.")
                control.should_training_stop = True

# --- 2. TRAINER AYARLARI ---
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        # Batch size'ı 2'de tutuyoruz ama grad. accum. düşürüyoruz.
        per_device_train_batch_size = 2,

        # HER 2 ADIMDA BİR GÜNCELLEME (Daha sıkı öğrenme)
        gradient_accumulation_steps = 2,

        # Bu ayarlarla 3 Epoch yetmeyebilir, 5 Epoch yapalım.
        # Callback olduğu için erken biterse sorun yok.
        num_train_epochs = 5,

        warmup_steps = 10,
        learning_rate = 2e-4, # Hızı koruyoruz
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
    callbacks=[TargetLossCallback()] # 0.38 - 0.40 hedefli callback aktif kalsın
)

print(f"Toplam Vaka: 513")
print(f"Bir adımda görülen vaka: {2*8} (Batch x GradAccum)")
print(f"Tüm veriyi 1 kez görmek için gereken adım: ~32")
print("Hedef: Tüm veriyi gör, Loss ~0.4 olunca dur.\n")

trainer_stats = trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/513 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


Toplam Vaka: 513
Bir adımda görülen vaka: 16 (Batch x GradAccum)
Tüm veriyi 1 kez görmek için gereken adım: ~32
Hedef: Tüm veriyi gör, Loss ~0.4 olunca dur.



==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 513 | Num Epochs = 5 | Total steps = 645
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 167,772,160 of 8,198,033,408 (2.05% trained)
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


...

wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.958100
2,1.957900
3,1.974800
4,1.877300
5,2.002400
6,1.860900
7,1.773200
8,1.714600
9,1.755300
10,1.636300



✋ HEDEF LOSS'A ULAŞILDI: 0.3569. Ezberi önlemek için durduruluyor.


train/epoch,▁▁▂▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇██
train/global_step,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇████
train/grad_norm,▁▄▅█▇▄▃▄▂▃▄▃▄▅▆▃▄▆▄▆▅▂▃▄▄▃▂▃▃▂▄▂▁▃▃▃▃▃▄▃
train/learning_rate,▁▂▃▅▆█████████████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆
train/loss,██▇▇▅▄▄▄▃▃▂▃▃▂▂▂▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▁▁
total_flos,2.368257673243853e+16
train/epoch,1.03113
train/global_step,133
train/grad_norm,0.28015
train/learning_rate,0.00016
train/loss,0.3569


In [ ]:
# Modeli GGUF formatında kaydet (q4_k_m = 4-bit orta kalite, dengeli)
print("Model GGUF formatına çevriliyor. Bu işlem birkaç dakika sürebilir...")
model.save_pretrained_gguf("model_medsim_tr", tokenizer, quantization_method = "q4_k_m")

print("Dönüştürme tamamlandı!")
print("Şimdi sol menüde 'model_medsim_tr' klasörü içindeki .gguf dosyasını indirebilirsin.")

Model GGUF formatına çevriliyor. Bu işlem birkaç dakika sürebilir...
Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [00:00<00:00, 14488.10it/s]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [00:58<00:00, 14.50s/it]


Unsloth: Merge process complete. Saved to `/content/model_medsim_tr`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF bf16 might take 3 minutes.
\        /    [2] Converting GGUF bf16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: llama.cpp found in the system. Skipping installation.
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into bf16 GGUF format.
This might take 3 minutes...


RuntimeError: Unsloth: GGUF conversion failed: Unsloth: Failed to convert model to GGUF: Command 'python llama.cpp/unsloth_convert_hf_to_gguf.py --outfile Meta-Llama-3.1-8B-Instruct.BF16.gguf --outtype bf16 --split-max-size 50G model_medsim_tr' returned non-zero exit status 1.

In [ ]:
# 1. Önce sadece eğitilmiş katmanları (LoRA) kaydet.
# Bu işlem çok hızlıdır ve hata verme ihtimali düşüktür.
model.save_pretrained("medsim_vaka_lora_backup")
tokenizer.save_pretrained("medsim_vaka_lora_backup")
print("✅ Güvenlik yedeği alındı! Eğitiminiz güvende.")

✅ Güvenlik yedeği alındı! Eğitiminiz güvende.


In [ ]:
from unsloth import FastLanguageModel
from google.colab import drive
drive.mount('/content/drive')

# LoRA yedeğini aldığınız yer
lora_yolu = "/content/medsim_vaka_lora_backup"
# 16-bit (Safetensors) olarak kaydedilecek yer
merged_kayit_yolu = "/content/drive/MyDrive/medsim_doktor_16bit_MERGED"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = lora_yolu,
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

print("Model birleştiriliyor ve Drive'a aktarılıyor (GGUF değil, Safetensors)...")
# GGUF yapma, sadece birleştir ve kaydet
model.save_pretrained_merged(
    merged_kayit_yolu,
    tokenizer,
    save_method = "merged_16bit", # Veya RAM çok darsa "merged_4bit"
)
print("✅ Adım 1 Tamam! Şimdi Runtime > Restart yapıp Adım 2'ye geç.")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.11.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Model birleştiriliyor ve Drive'a aktarılıyor (GGUF değil, Safetensors)...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [01:01<00:00, 15.30s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [01:25<00:00, 21.41s/it]


Unsloth: Merge process complete. Saved to `/content/drive/MyDrive/medsim_doktor_16bit_MERGED`
✅ Adım 1 Tamam! Şimdi Runtime > Restart yapıp Adım 2'ye geç.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Aracı indir
!git clone https://github.com/ggerganov/llama.cpp
!pip install -r llama.cpp/requirements.txt

# Ayarlar
input_klasor = "/content/drive/MyDrive/medsim_doktor_16bit_MERGED"
output_dosya = "/content/drive/MyDrive/medsim_doktor_FINAL.gguf"

print("Dönüşüm Başlıyor...")
# Bu script modeli RAM'e yüklemez, diskten diske okur/yazar. Çökme ihtimali %0'dır.
!python llama.cpp/convert_hf_to_gguf.py {input_klasor} --outfile {output_dosya} --outtype q4_k_m

print("🎉 MUTLU SON! Dosyanız hazır.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path 'llama.cpp' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly
Ignoring torch: markers 'platform_machine == "s390x"' don't match your environment
Ignoring torch: markers 'platform_machine == "s390x"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 129.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 129.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 

Dönüşüm Başlıyor...
usage: convert_hf_to_gguf.py [-h] [--vocab-only] [--outfile OUTFILE]
                             [--outtype {f32,f16,bf16,q8_0,tq1_0,tq2_0,auto}]
                             [--bigendian] [--use-temp-file] [--no-lazy]
                             [--model-name MODEL_NAME] [--verbose]
                             [--split-max-tensors SPLIT_MAX_TENSORS]
                             [--split-max-size SPLIT_MAX_SIZE] [--dry-run]
                             [--no-tensor-first-split] [--metadata METADATA]
                             [--print-supported-models] [--remote] [--mmproj]
                             [--mistral-format]
                             [--disable-mistral-community-chat-template]
                             [--sentence-transformers-dense-modules]
                             [model]
convert_hf_to_gguf.py: error: argument --outtype: invalid choice: 'q4_k_m' (choose from f32, f16, bf16, q8_0, tq1_0, tq2_0, auto)
🎉 MUTLU SON! Dosyanız hazır.


In [ ]:
# 1. Drive'ı Bağla
from google.colab import drive
drive.mount('/content/drive')

import os

# --- AYARLAR ---
input_klasor = "/content/drive/MyDrive/medsim_doktor_16bit_MERGED"
temp_f16_dosya = "/content/drive/MyDrive/temp_medsim_f16.gguf"
final_dosya = "/content/drive/MyDrive/medsim_doktor_FINAL_Q4.gguf"

# ---------------------------------------------------------
print("🛠️ ADIM 1: Sıkıştırma Motoru Derleniyor (CMake ile)...")

# CMake ve gerekli araçları kur
!apt-get update -qq && apt-get install -y cmake build-essential > /dev/null

# llama.cpp'yi temiz çek
if os.path.exists("llama.cpp"):
    import shutil
    shutil.rmtree("llama.cpp")

!git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp
!mkdir build
%cd build
# Yeni derleme sistemi
print("Derleme başlıyor (1-2 dk sürebilir)...")
!cmake .. > /dev/null
!cmake --build . --config Release -j 4 > /dev/null
%cd ../..

# Binary dosyanın tam yolu
quantize_bin = "./llama.cpp/build/bin/llama-quantize"

# ---------------------------------------------------------
print("\n🔍 ADIM 2: F16 Dosyası Kontrol Ediliyor...")

# Eğer önceki kod 16GB'lık dosyayı sildiyse (ki temizlik kısmında silmiş olabilir), tekrar oluşturuyoruz.
if not os.path.exists(temp_f16_dosya):
    print("⚠️ F16 dosyası bulunamadı (önceki adımda silinmiş), tekrar oluşturuluyor...")
    # Transformers sürümü zaten düzgün olduğu için direkt çalışır
    !python llama.cpp/convert_hf_to_gguf.py {input_klasor} --outfile {temp_f16_dosya} --outtype f16
else:
    print("✅ F16 dosyası zaten var, dönüştürme atlanıyor.")

# ---------------------------------------------------------
print("\n📦 ADIM 3: Sıkıştırma (Q4_K_M)...")

if os.path.exists(quantize_bin) and os.path.exists(temp_f16_dosya):
    # Sıkıştırma komutu
    !{quantize_bin} {temp_f16_dosya} {final_dosya} q4_k_m

    print("\n🧹 Temizlik...")
    if os.path.exists(temp_f16_dosya):
        os.remove(temp_f16_dosya)
        print("Geçici 16GB dosya silindi, Drive'da yer açıldı.")

    print(f"\n🎉🎉 MUTLU SON! İŞTE DOSYANIZ: {final_dosya}")
    print("Artık bu dosyayı indirip kullanabilirsiniz.")
else:
    print("\n❌ HATA: Derleme başarısız oldu veya dosya bulunamadı.")
    print(f"Binary yol: {quantize_bin}")
    print(f"F16 Dosya: {os.path.exists(temp_f16_dosya)}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🛠️ ADIM 1: Sıkıştırma Motoru Derleniyor (CMake ile)...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Cloning into 'llama.cpp'...
remote: Enumerating objects: 69565, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 69565 (delta 26), reused 15 (delta 14), pack-reused 69524 (from 2)
Receiving objects: 100% (69565/69565), 211.38 MiB | 13.11 MiB/s, done.
Resolving deltas: 100% (50421/50421), done.
/content/llama.cpp
/content/llama.cpp/build
Derleme başlıyor (1-2 dk sürebilir)...
CMAKE_BUILD_TYPE=Release
/content

🔍 ADIM 2: F16 Dosyası Kontrol Ediliyor...
⚠️ F16 dosyası bulunamadı (önceki adımda silinmiş), tekrar oluşturuluyor...
INFO:hf-to-gguf:Loading model:

In [ ]:
# 1. GGUF Çalıştırma Kütüphanesi Kuruluyor (Çok hızlı)
!pip install llama-cpp-python

# 2. Drive Bağlanıyor
from google.colab import drive
drive.mount('/content/drive')

from llama_cpp import Llama
import sys

# 3. Model Yükleniyor
# Önceki adımda oluşturduğumuz dosya yolu
model_path = "/content/drive/MyDrive/medsim_doktor_FINAL_Q4.gguf"

print(f"\n📂 Model yükleniyor: {model_path}")
print("RAM'e alınıyor, lütfen bekleyin...")

try:
    # Model başlatılıyor
    llm = Llama(
        model_path=model_path,
        n_ctx=8192,       # Llama 3.1 için geniş hafıza
        n_gpu_layers=-1,  # GPU varsa kullan
        verbose=False
    )
    print("✅ MODEL BAŞARIYLA YÜKLENDİ! Dosya sağlam.")

    # 4. Test Sorusu
    print("\n--- TEST SORUSU SORULUYOR ---")
    soru = """Tıp eğitimi için bir vaka oluştur. Vakalar her türlü branştan gelebilir.
                    SİSTEM TALİMATI (System Instruction):
                    1. ROLÜN: [Hasta. İsim, Yaş, Cinsiyet, Meslek].
                    2. ŞİKAYET: [Tek cümlelik, net giriş şikayeti].
                    3. GERÇEK TANI: [Gizli].
                    4. TARZIN: Tamamen hasta rolündesin. "Yapay zekayım" deme. Yapay zekanın bilgilendirici fonksiyonlarını asla kullanma. Tam bir hasta profili olarak davran.
                       - Konuşman "motomot", kısa ve net olsun. Duygusallık yok.
                       - Sadece sorulan soruya cevap ver.
                       - Örnek: "Ağrın nerede?" -> "Göğsümde." (Hikaye anlatma).
                    5. TIBBİ VERİLER (Doktor özel olarak isterse parantez içinde teknik dille ver):
                       - İstenmeyen veriyi asla verme.
                    6. ORDER/TEDAVİ: Doktor order girdiğinde (İlaç, doz), uygunluğunu teknik dille değerlendir.
  """
    prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nSen uzman bir doktorsun. JSON ver.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n{soru}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

    output = llm(prompt, max_tokens=500, stop=["<|eot_id|>"], echo=False)
    print("DOKTORUN CEVABI:")
    print(output['choices'][0]['text'])

except Exception as e:
    print(f"\n❌ HATA: Model Colab'de bile çalışmadı. Dosya oluşturulurken bozulmuş.")
    print(f"Hata detayı: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

📂 Model yükleniyor: /content/drive/MyDrive/medsim_doktor_FINAL_Q4.gguf
RAM'e alınıyor, lütfen bekleyin...


llama_context: n_ctx_per_seq (8192) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


✅ MODEL BAŞARIYLA YÜKLENDİ! Dosya sağlam.

--- TEST SORUSU SORULUYOR ---
DOKTORUN CEVABI:
### ROLÜ: Emre Can, 45 yaşında, Erkek, Mühendis.
### ŞİKAYET: Son zamanlarda sürekli yorgunluk ve karın ağrısı şikayetim var.
### GERÇEK TANI: Akut Pankreatit.
### TARZIN: Tamamen hasta rolündeyim. Yapay zekam yok.

Doktor: Hasta Emre Can Bey, size acil servis doktoru Dr. Demir'in doktoru. Şikayetlerinizi lütfen özetleyiniz.

Ben: Doktor bey, son 2 gündür sürekli yorgunluk ve midem bulanıyor. Karın ağrım var, özellikle üst kadranda. Geçen hafta midemde biraz rahatsızlık oldum ama geçmişti. Şimdi ise daha kötü.

Doktor: Anında acil servise gelen bir hasta. Akut pankreatit düşündürür. Nedenini bilmek için tansiyonunuzu, kan şekerinizi ve karın ağrısına bağlı olarak ağrı kesicilerinizi kullandığını lütfen belirtin.

Ben: Tansiyonum 130/80, kan şekeri 105 mg/dL. Son birkaç gündür karın ağrım için parasetamol 650 mg 2 tablet almıştım ama bir fayda etmedi.

Doktor: Tamam, acil servise refleks tarafa (AC

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
# 1. Google Drive'ı Bağla
from google.colab import drive
drive.mount('/content/drive')

import os
import subprocess
import time
import threading
import json

# --- AYARLAR ---
DRIVE_PATH = "/content/drive/MyDrive"
MODEL_PATH = f"{DRIVE_PATH}/medsim_doktor_FINAL_Q4.gguf"
TOOL_PATH = f"{DRIVE_PATH}/cloudflared-linux-amd64"
KNOWLEDGE_FILE = f"{DRIVE_PATH}/medsim_hafiza.json" # Hafıza dosyası
MODEL_NAME = "medsim"

# --- 1. CLOUDFLARE KONTROL (HIZLI BAŞLATMA) ---
if not os.path.exists(TOOL_PATH):
    print("📥 Cloudflare indiriliyor...")
    !wget -q -O {TOOL_PATH} https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
    !chmod +x {TOOL_PATH}
else:
    print("⚡ Cloudflare Drive'dan kullanılıyor.")

!cp {TOOL_PATH} ./cloudflared
!chmod +x ./cloudflared

# --- 2. KURULUMLAR ---
print("⏳ Kütüphaneler kuruluyor...")
!curl -fsSL https://ollama.com/install.sh | sh > /dev/null
!pip install streamlit ollama pyngrok rapidfuzz > /dev/null

# --- 3. OLLAMA BAŞLAT ---
def start_ollama():
    subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

thread = threading.Thread(target=start_ollama)
thread.start()
time.sleep(5)

# --- 4. MODELİ TANIT (KATİ TÜRKÇE) ---
print(f"🛠️ '{MODEL_NAME}' modeli hazırlanıyor...")
modelfile_content = f"""
FROM {MODEL_PATH}
SYSTEM "Sen bir hastasın. Doktorla konuşuyorsun.
KURALLAR:
1. Sadece TÜRKÇE konuş.
2. Kısa ve net cevap ver. Doktor sormadan detay verme. Pasif ol.
3. Tıbbi terim kullanma. Halk ağzıyla konuş.
4. Asla JSON formatı kullanma."
PARAMETER temperature 0.8
"""
with open("Modelfile", "w") as f:
    f.write(modelfile_content)

!ollama create {MODEL_NAME} -f Modelfile
print("✅ Model hazır!")

# --- 5. STREAMLIT UYGULAMASI (HER ŞEY DAHİL) ---
app_code = f"""
import streamlit as st
import ollama
import json
import os
from rapidfuzz import process, fuzz
import time

# --- HAFIZA FONKSİYONLARI ---
KNOWLEDGE_DB = "{KNOWLEDGE_FILE}"

def load_db():
    if os.path.exists(KNOWLEDGE_DB):
        with open(KNOWLEDGE_DB, 'r', encoding='utf-8') as f:
            return json.load(f)
    return {{}}

def save_to_db(soru, cevap):
    db = load_db()
    db[soru] = cevap
    with open(KNOWLEDGE_DB, 'w', encoding='utf-8') as f:
        json.dump(db, f, ensure_ascii=False, indent=4)

def check_memory(soru):
    db = load_db()
    if not db: return None
    keys = list(db.keys())
    match = process.extractOne(soru, keys, scorer=fuzz.token_sort_ratio)
    if match and match[1] >= 85: # %85 Benzerlik eşiği
        return db[match[0]]
    return None

# --- SAYFA AYARLARI ---
st.set_page_config(page_title="MedSim Pro", page_icon="🩺", layout="wide", initial_sidebar_state="expanded")

# --- CSS (RENK VE SABİT MENÜ DÜZELTMESİ) ---
st.markdown(\"\"\"
<style>
    /* Yan menü kapatma okunu gizle */
    [data-testid="stSidebarCollapsedControl"] {{ display: none !important; }}

    /* Sol Paneli Sabitle ve Renklendir */
    [data-testid="stSidebar"] {{
        background-color: #f8f9fa !important;
        border-right: 1px solid #dee2e6;
        min-width: 350px !important;
        max-width: 350px !important;
    }}

    /* Tüm yazıları siyah yap (Light Mode zorlaması için ek) */
    .stApp, [data-testid="stSidebar"] * {{ color: black !important; }}

    /* Seçim kutularını düzelt */
    .stMultiSelect, .stSelectbox, .stTextInput {{ color: black !important; background-color: white !important; }}
    span[data-baseweb="tag"] {{ background-color: #e3f2fd !important; color: #000000 !important; }}

    /* Chat Input Sabitleme */
    [data-testid="stChatInput"] {{
        position: fixed; bottom: 0; left: 350px; right: 0; padding: 20px;
        background: white; z-index: 999; border-top: 2px solid #eee;
    }}
    @media (max-width: 768px) {{ [data-testid="stChatInput"] {{ left: 0; }} }}

    .main .block-container {{ padding-bottom: 150px; }}

    /* Butonlar */
    .stButton button {{ width: 100%; border-radius: 8px; font-weight: bold; border: 1px solid #ccc; }}
    .stButton button:hover {{ border-color: #0d6efd; color: #0d6efd; }}

    /* Öğretmen Paneli Stili */
    .teacher-box {{ border: 2px solid #ff4b4b; padding: 10px; border-radius: 10px; background-color: #fff0f0; margin-bottom: 20px; }}
</style>
\"\"\", unsafe_allow_html=True)

if "history" not in st.session_state:
    st.session_state.history = []

# --- YAN MENÜ ---
with st.sidebar:
    st.title("📋 Kontrol Paneli")

    if st.button("🎲 YENİ VAKA BAŞLAT", type="primary"):
        st.session_state.history = []
        with st.spinner("Hasta hazırlanıyor..."):
            try:
                res = ollama.chat(model='medsim', messages=[{{'role': 'user', 'content': "Senaryoyu başlat. Sen bir hastasın. Sadece 'Merhaba doktor bey/hanım, kendimi kötü hissediyorum' de."}}])
                st.session_state.history.append({{'role': 'assistant', 'content': res['message']['content']}})
                st.rerun()
            except: pass

    st.markdown("---")

    # --- ÖĞRETMEN PANELİ (YENİ) ---
    with st.expander("🎓 ÖĞRETMEN MODU (Hafıza)", expanded=True):
        st.info("Model yanlış cevap verdiyse düzeltip kaydedin.")
        last_user = ""
        last_bot = ""
        if len(st.session_state.history) >= 2:
            for msg in reversed(st.session_state.history):
                if msg['role'] == 'assistant' and not last_bot: last_bot = msg['content']
                if msg['role'] == 'user' and not last_user: last_user = msg['content']
                if last_user and last_bot: break

        if last_user and last_bot and "[" not in last_user: # Teknik mesaj değilse
            with st.form("teacher_form"):
                st.write(f"**Soru:** {{last_user}}")
                correct_answer = st.text_area("Doğrusunu Yaz:", value=last_bot)
                if st.form_submit_button("💾 Hafızaya Kaydet"):
                    save_to_db(last_user, correct_answer)
                    st.success("Kaydedildi!")
                    time.sleep(1)

    st.markdown("---")

    # 1. FİZİK MUAYENE (Seçimli)
    st.markdown("##### 🩺 Fizik Muayene")
    fm_secenekler = ["Genel Durum", "Vital Bulgular", "Baş-Boyun", "Solunum", "Kardiyovasküler", "Batın", "Nörolojik", "Ekstremiteler", "Deri"]
    fm_secim = st.multiselect("Bölge Seçin:", fm_secenekler, key="fm_key", label_visibility="collapsed")

    if st.button("Seçilenleri Muayene Et"):
        if fm_secim:
            st.session_state.history.append({{'role': 'user', 'content': f"📝 [Muayene]: {{', '.join(fm_secim)}}"}})
            tech_prompt = f"Sen teknik bir tıbbi raporsun. Şu sistemlerin muayene bulgularını teknik dille yaz: {{', '.join(fm_secim)}}. Başka sistem yazma."

            with st.spinner("Muayene ediliyor..."):
                msgs = st.session_state.history.copy() + [{{'role': 'user', 'content': tech_prompt}}]
                res = ollama.chat(model='medsim', messages=msgs, options={{'temperature': 0.1}})
                st.session_state.history.append({{'role': 'assistant', 'content': res['message']['content']}})
                st.rerun()
        else:
            st.error("Bölge seçmediniz.")

    st.markdown("---")

    # 2. LABORATUVAR (Seçimli)
    st.markdown("##### 🧪 Laboratuvar")
    lab_secenekler = ["Hemogram", "Biyokimya", "Elektrolitler", "Karaciğer Fonk.", "Böbrek Fonk.", "Troponin", "INR", "Kan Gazı", "D-Dimer", "TIT"]
    lab_secim = st.multiselect("Tetkik Seçin:", lab_secenekler, key="lab_key", label_visibility="collapsed")

    if st.button("Tetkikleri İste"):
        if lab_secim:
            st.session_state.history.append({{'role': 'user', 'content': f"📝 [Lab]: {{', '.join(lab_secim)}}"}})
            tech_prompt = f"Şu tetkiklerin sonuçlarını raporla: {{', '.join(lab_secim)}}. Referans aralığı verme, sadece sonucu yaz."

            with st.spinner("Sonuçlar çıkıyor..."):
                msgs = st.session_state.history.copy() + [{{'role': 'user', 'content': tech_prompt}}]
                res = ollama.chat(model='medsim', messages=msgs, options={{'temperature': 0.1}})
                st.session_state.history.append({{'role': 'assistant', 'content': res['message']['content']}})
                st.rerun()
        else:
            st.error("Tetkik seçmediniz.")

    st.markdown("---")

    # 3. GÖRÜNTÜLEME
    st.markdown("##### 🩻 Görüntüle")
    col1, col2 = st.columns([1, 2])
    with col1:
        goruntu_tur = st.selectbox("Tür:", ["Röntgen", "BT", "USG", "MR", "EKG"], key="rad_mod", label_visibility="collapsed")
    with col2:
        goruntu_bolge = st.text_input("Bölge:", placeholder="Örn: Akciğer", key="rad_area", label_visibility="collapsed")

    if st.button("Görüntüle"):
        if goruntu_bolge:
            istek = f"{{goruntu_bolge}} {{goruntu_tur}}"
            st.session_state.history.append({{'role': 'user', 'content': f"📝 [Radyoloji]: {{istek}} çekildi."}})
            tech_prompt = f"Radyoloji raporu yaz. İstenen: {{istek}}. Bulguları teknik dille yaz."

            with st.spinner("Görüntüleniyor..."):
                msgs = st.session_state.history.copy() + [{{'role': 'user', 'content': tech_prompt}}]
                res = ollama.chat(model='medsim', messages=msgs, options={{'temperature': 0.1}})
                st.session_state.history.append({{'role': 'assistant', 'content': res['message']['content']}})
                st.rerun()
        else:
            st.error("Bölge giriniz.")

# --- ANA EKRAN ---
st.title("🩺 MedSim")

for message in st.session_state.history:
    role = message["role"]
    content = message["content"]
    if role == "system": continue

    if role == "user":
        if "[" in content: st.info(content)
        else:
            with st.chat_message("user", avatar="🧑‍⚕️"): st.markdown(content)
    elif role == "assistant":
        with st.chat_message("assistant", avatar="👤"):
            # Temizlik ve Hafıza Göstergesi
            clean = content.replace('{', '').replace('}', '').replace('"anamnez":', '')
            if "(🧠 Hatırladım)" in clean:
                st.markdown(f"{{clean}}")
                st.caption("✨ *Hafızadan çekildi*")
            else:
                st.markdown(clean)

prompt = st.chat_input("Hastaya soru sorun...")

if prompt:
    st.session_state.history.append({{"role": "user", "content": prompt}})
    with st.chat_message("user", avatar="🧑‍⚕️"): st.markdown(prompt)

    with st.chat_message("assistant", avatar="👤"):
        with st.spinner("Cevap veriliyor..."):

            # 1. HAFIZA KONTROLÜ
            memory_reply = check_memory(prompt)

            if memory_reply:
                bot_reply = memory_reply + " (🧠 Hatırladım)"
                time.sleep(0.3)
            else:
                # 2. MODEL CEVABI
                msgs = [{{'role': 'system', 'content': "Sen bir hastasın. Kısa ve net konuş. Tıbbi terim bilmezsin."}}] + st.session_state.history
                res = ollama.chat(model='medsim', messages=msgs, options={{'temperature': 0.8}})
                bot_reply = res['message']['content']

            st.markdown(bot_reply)

    st.session_state.history.append({{"role": "assistant", "content": bot_reply}})
"""

with open("app.py", "w") as f:
    f.write(app_code)

# --- BAŞLAT (ZORLA LIGHT MODE) ---
print("\n🌍 UYGULAMA BAŞLATILIYOR...")
print("Lütfen 'trycloudflare.com' linkine tıkla.\n")

subprocess.Popen(["streamlit", "run", "app.py", "--server.enableCORS=false", "--server.enableXsrfProtection=false", "--server.port=8501", "--theme.base='light'"])
!./cloudflared tunnel --url http://localhost:8501

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
⚡ Cloudflare Drive'dan kullanılıyor.
⏳ Kütüphaneler kuruluyor...
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
🛠️ 'medsim' modeli hazırlanıyor...

✅ Model hazır!

🌍 UYGULAMA BAŞLATILIYOR...
Lütfen 'trycloudflare.com' linkine tıkla.

2025-11-29T20:23:35Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to th